In [7]:
import pandas as pd

# Load required datasets
orders = pd.read_csv('C:/Users/bhavi/Desktop/delivery-delay-prediction/data/raw/orders_dataset.csv')
order_items = pd.read_csv('C:/Users/bhavi/Desktop/delivery-delay-prediction/data/raw/order_items_dataset.csv')
customers = pd.read_csv('C:/Users/bhavi/Desktop/delivery-delay-prediction/data/raw/customers_dataset.csv')
sellers = pd.read_csv('C:/Users/bhavi/Desktop/delivery-delay-prediction/data/raw/sellers_dataset.csv')
products = pd.read_csv('C:/Users/bhavi/Desktop/delivery-delay-prediction/data/raw/products_dataset.csv')

# Merge datasets
df = orders.merge(order_items, on='order_id', how='left')
df = df.merge(customers[['customer_id', 'customer_state']], on='customer_id', how='left')
df = df.merge(sellers[['seller_id', 'seller_state']], on='seller_id', how='left')
df = df.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')

# Convert dates
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

# Remove rows with missing delivery dates
df = df.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date'])

# Create target column: is_late
df['is_late'] = (df['order_delivered_customer_date'] > df['order_estimated_delivery_date']).astype(int)

# Feature engineering
df['shipping_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
df['estimated_days'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.days

# Final columns
final_df = df[[
    'product_category_name',
    'customer_state',
    'seller_state',
    'shipping_days',
    'estimated_days',
    'is_late'
]]

final_df.to_csv('../data/processed/final_merged_data.csv', index=False)
final_df.head()


,product_category_name,customer_state,seller_state,shipping_days,estimated_days,is_late
0,utilidades_domesticas,SP,SP,8,15,0
1,perfumaria,BA,SP,13,19,0
2,automotivo,GO,SP,9,26,0
3,pet_shop,RN,MG,13,26,0
4,papelaria,SP,SP,2,12,0
